In [8]:
import os
import numpy as np                        # Fundamental package for scientific computing
import matplotlib.pyplot as plt           # 2D plotting library producing publication quality figures
from IPython.display import clear_output  # Clear the screen
import pyrealsense2 as rs                 # Intel RealSense cross-platform open-source API
print("Environment is Ready")
from plyfile import PlyData


Environment is Ready


Single slide dpeth visualization

In [ ]:
pipe = rs.pipeline()                      # Create a pipeline
cfg = rs.config()                         # Create a default configuration
print("Pipeline is created")

In [ ]:
print("Searching Devices..")
selected_devices = []                     # Store connected device(s)

for d in rs.context().devices:
    selected_devices.append(d)
    print(d.get_info(rs.camera_info.name))
if not selected_devices:
    print("No RealSense device is connected!")

In [ ]:
rgb_sensor = depth_sensor = None

for device in selected_devices:                         
    print("Required sensors for device:", device.get_info(rs.camera_info.name))
    for s in device.sensors:                              # Show available sensors in each device
        if s.get_info(rs.camera_info.name) == 'RGB Camera':
            print(" - RGB sensor found")
            rgb_sensor = s                                # Set RGB sensor
        if s.get_info(rs.camera_info.name) == 'Stereo Module':
            depth_sensor = s                              # Set Depth sensor
            print(" - Depth sensor found")

In [ ]:
colorizer = rs.colorizer()                                # Mapping depth data into RGB color space
profile = pipe.start(cfg)                                 # Configure and start the pipeline

fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(12,4)) # Show 1 row with 2 columns for Depth and RGB frames
title = ["Depth Image", "RGB Image"]                      # Title for each frame

for _ in range(10):                                       # Skip first frames to give syncer and auto-exposure time to adjust
    frameset = pipe.wait_for_frames()
    
for _ in range(5):                                        # Increase to display more frames
    frameset = pipe.wait_for_frames()                     # Read frames from the file, packaged as a frameset
    depth_frame = frameset.get_depth_frame()              # Get depth frame
    color_frame = frameset.get_color_frame()              # Get RGB frame

    colorized_streams = []                                # This is what we'll actually display
    if depth_frame:
        colorized_streams.append(np.asanyarray(colorizer.colorize(depth_frame).get_data()))
    if color_frame:
        colorized_streams.append(np.asanyarray(color_frame.get_data()))
    
    for i, ax in enumerate(axs.flatten()):                # Iterate over all (Depth and RGB) colorized frames
        if i >= len(colorized_streams): continue          # When getting less frames than expected
        plt.sca(ax)                                       # Set the current Axes and Figure
        plt.imshow(colorized_streams[i])                  # colorized frame to display
        plt.savefig("./depth435/output.jpg")
        plt.title(title[i])                               # Add title for each subplot
    clear_output(wait=True)                               # Clear any previous frames from the display
    plt.tight_layout()                                    # Adjusts display size to fit frames
    plt.pause(1)                                          # Make the playback slower so it's noticeable
    
pipe.stop()                                               # Stop the pipeline
print("Done!")

Capture pcd and save .ply format or raw

In [67]:
#csv writer
import csv
from datetime import datetime
header = [
    "datetime",
    "frame_number",
    "x",
    "y",
    "z"
    
]
filename = "data.csv"  
with open(filename, "w") as f:
    csv.DictWriter(f, fieldnames=header).writeheader()

In [68]:
import pyrealsense2 as rs
import numpy as np
import open3d as o3d
import cv2

# Configure depth and color streams
pipeline = rs.pipeline()
config = rs.config()

# Enable depth and color streams
config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)

# Start streaming
pipeline.start(config)

# Create an align object
# rs.align allows us to perform alignment of depth frames to others frames
align_to = rs.stream.color
align = rs.align(align_to)

try:
    while True:
        # Wait for a coherent pair of frames: depth and color
        frames = pipeline.wait_for_frames()
        
        # Align the depth frame to color frame
        aligned_frames = align.process(frames)
        
        # Get aligned frames
        depth_frame = aligned_frames.get_depth_frame()
        color_frame = aligned_frames.get_color_frame()

        if not depth_frame or not color_frame:
            continue

        # Convert images to numpy arrays
        depth_image = np.asanyarray(depth_frame.get_data())
        color_image = np.asanyarray(color_frame.get_data())

        # Create point cloud object
        pc = rs.pointcloud()
        points = pc.calculate(depth_frame)
        pc.map_to(color_frame)
        # print(points.get_vertices())

        dict_dumper = {'datetime': datetime.now()}
        data = {
            "datetime" : datetime.now(),
            "frame_number": points.get_frame_number(),
            "x":np.asanyarray(points.get_vertices())['f0'],
            "y":np.asanyarray(points.get_vertices())['f1'],
            "z":np.asanyarray(points.get_vertices())['f2']
        }
        
        dict_dumper.update(data)
        path = "./data.csv"
        with open(path, "a") as f:
            writer = csv.DictWriter(f, header)
            writer.writerow(dict_dumper)
        # print("Frame #:",points.get_frame_number())
        # print("Frame size:", points.get_data_size())
        # print("Frame profile:", points.profile)
        # # depth = np.asanyarray(points.get_data())
        # depth = np.asanyarray(points.as_points().get_data()) 
 
        # print("Depth shape", depth.shape)
        # print("Depth", depth)  


        # Save point cloud data as a .ply file
        # points.export_to_ply("point_cloud.ply", color_frame)
        # print("Saved point cloud to 'point_cloud.ply'")

        # Display the color image
        cv2.imshow('RealSense', color_image)

        # Exit on 'q' key press
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

finally:
    # Stop streaming
    pipeline.stop()

    # Close OpenCV windows
    cv2.destroyAllWindows()


KeyboardInterrupt: 

In [73]:
np.asanyarray(points.as_pose_frame())


array(<pyrealsense2.frame NULL>, dtype=object)

Visualize captured pcd data

In [1]:
import open3d as o3d

# Load the .ply file
ply_file = "point_cloud.ply"
pcd = o3d.io.read_point_cloud(ply_file)

# Print basic information about the point cloud
print(pcd)

# Visualize the point cloud
o3d.visualization.draw_geometries([pcd], window_name="Point Cloud Viewer", width=800, height=600, left=50, top=50)


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
PointCloud with 266589 points.


Open captured pcd data

In [5]:
plydata = PlyData.read(ply_file)

In [7]:
# Extract vertex data
vertex_data = plydata['vertex'].data

# Convert to numpy array
points = np.vstack([vertex_data['x'], vertex_data['y'], vertex_data['z']]).T

# Extract colors if available
colors = None
if 'red' in vertex_data.dtype.names:
    colors = np.vstack([vertex_data['red'], vertex_data['green'], vertex_data['blue']]).T

# Extract normals if available
normals = None
if 'nx' in vertex_data.dtype.names:
    normals = np.vstack([vertex_data['nx'], vertex_data['ny'], vertex_data['nz']]).T

# Print the extracted data
print("Points:")
print(points)
if colors is not None:
    print("Colors:")
    print(colors)
if normals is not None:
    print("Normals:")
    print(normals)

# Save extracted data to files if needed
np.savetxt("points.csv", points, delimiter=",")
if colors is not None:
    np.savetxt("colors.csv", colors, delimiter=",")
if normals is not None:
    np.savetxt("normals.csv", normals, delimiter=",")

Points:
[[-2.140079    1.55787    -4.03      ]
 [-2.133443    1.55787    -4.03      ]
 [-2.126807    1.55787    -4.03      ]
 ...
 [ 0.23667763 -0.18376403 -0.45700002]
 [ 0.23898879 -0.18497035 -0.46      ]
 [ 0.23974624 -0.18497035 -0.46      ]]
Colors:
[[ 77 107 110]
 [ 77 107 110]
 [ 77 107 110]
 ...
 [ 77 107 110]
 [ 77 107 110]
 [ 77 107 110]]


VISUALIZE EXISTING CAPTURE


In [ ]:
!wget -N https://librealsense.intel.com/rs-tests/TestData/stairs.bag
print("Data is Ready")


In [ ]:
# Setup:
pipe = rs.pipeline()
cfg = rs.config()
cfg.enable_device_from_file("stairs.bag")
profile = pipe.start(cfg)

In [ ]:
##capture depth frame

# Skip 5 first frames to give the Auto-Exposure time to adjust
for x in range(5):
  pipe.wait_for_frames()
  
# Store next frameset for later processing:
frameset = pipe.wait_for_frames()
depth_frame = frameset.get_depth_frame()

# Cleanup:
pipe.stop()
print("Frames Captured")


In [ ]:
colorizer = rs.colorizer()
colorized_depth = np.asanyarray(colorizer.colorize(depth_frame).get_data())

plt.rcParams["axes.grid"] = False
plt.rcParams['figure.figsize'] = [8, 4]
plt.imshow(colorized_depth)